In [18]:
from io import BytesIO
from urllib import request
import onnx
import numpy as np
from PIL import Image
import onnxruntime as ort
import math

In [3]:

model = onnx.load("hair_classifier_v1.onnx")
print("Inputs:", [i.name for i in model.graph.input])
print("Outputs:", [o.name for o in model.graph.output])


Inputs: ['input']
Outputs: ['output']


In [4]:
input_tensor = model.graph.input[0]  # first input
shape = [dim.dim_value for dim in input_tensor.type.tensor_type.shape.dim]
print(shape)


[0, 3, 200, 200]


In [15]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_image(img):
    arr = np.array(img).astype(np.float32)   # H x W x 3
    arr = arr / 255.0                         # scale to 0-1
    arr = (arr - 0.5) / 0.5                  # center to [-1, 1]
    return arr

img = download_image(url)
img_prepared = prepare_image(img, target_size=(200, 200))
arr = preprocess_image(img_prepared)

raw = np.array(img_prepared)[0, 0, 0]   # pixel before normalization
print("Shape:", arr.shape)
print("Raw first pixel R value:", raw)
print("After /255:", raw / 255.0)
print("Final normalized:", (raw / 255.0 - 0.5) / 0.5)



Shape: (200, 200, 3)
Raw first pixel R value: 61
After /255: 0.23921568627450981
Final normalized: -0.5215686274509803


In [19]:
def preprocess(img):
    x = np.array(img).astype(np.float32)
    x = x / 255.0
    x = (x - 0.5) / 0.5
    x = np.transpose(x, (2, 0, 1))  # HWC → CHW
    x = np.expand_dims(x, axis=0)   # add batch dimension
    return x

# --- Load image and preprocess ---
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img, (200, 200))
x = preprocess(img)

session = ort.InferenceSession("hair_classifier_v1.onnx")

inputs = {session.get_inputs()[0].name: x}
outputs = session.run(None, inputs)
logit = outputs[0][0]
prob = 1 / (1 + math.exp(-logit))
print(prob)


0.4498383638070855


C:\Users\rimsh\AppData\Local\Temp\ipykernel_5560\317362155.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob = 1 / (1 + math.exp(-logit))
